In [27]:
import numpy as np
import os
import cv2


DATADIR = "C:/Users/PC PRO ULTRA RAPIDE/Desktop/Projects/artificial intelligence/ml/sign-lang/sign-lang-dataset/"

ALPHABETS = [
    "ain", "al", "aleff", "ba", "dad", "dal", "dhad", "fa", "ghain", "ha",
    "haae", "jeem", "kaaf", "khaa", "la", "laam", "meem", "nun", "qaf", "ra", "saad",
    "seen", "sheen", "taae", "taae2", "thaae", "thdad", "waw", "ya", "zay" ]

train_ds = []

for alphabet in ALPHABETS:
    path = os.path.join(DATADIR, alphabet)
    class_num = ALPHABETS.index(alphabet)
    for img in os.listdir(path):        
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
        train_ds.append([img_array, class_num])

In [97]:
image_size = (64, 64)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATADIR,
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATADIR,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 51174 files belonging to 30 classes.
Using 40940 files for training.
Found 51174 files belonging to 30 classes.
Using 10234 files for validation.


In [98]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        #layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

In [99]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

In [105]:
print(len(train_ds))
print(len(val_ds))
print((len(train_ds)+len(val_ds))*32)

1280
320
51200


In [107]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    
    # Image augmentation block
    x = data_augmentation(inputs)

    # Entry block
    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes


    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)

model = make_model(input_shape= image_size + (3,), num_classes=30)

model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
sequential_10 (Sequential)      (None, 64, 64, 3)    0           input_17[0][0]                   
__________________________________________________________________________________________________
rescaling_7 (Rescaling)         (None, 64, 64, 3)    0           sequential_10[1][0]              
__________________________________________________________________________________________________
conv2d_42 (Conv2D)              (None, 32, 32, 32)   896         rescaling_7[0][0]                
____________________________________________________________________________________________

In [112]:
epochs = 1

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

1280/1280 [==============================] - 1484s 1s/step - loss: 0.8476 - accuracy: 0.7333 - val_loss: 0.7386 - val_accuracy: 0.7771


C:\Users\PC PRO ULTRA RAPIDE\anaconda3\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [ ]:
model.save('alphabet.model')  # save the model